# 드라이브 마운트, 라이브러리 Import, 데이터 셋 가져오기

In [1]:
#구글 드라이브 마운트 (구글 드라이브에 제공된 데이터 셋 업로드 후 이용)
from google.colab import drive
drive.mount('/content/drive')

In [2]:
#필요한 라이브러리 Import
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [ ]:
#데이터 셋 불러오기
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/소방안전AI예측/02.격자단위_구급출동_dataset.csv', encoding = 'euc-kr')

# 데이터 셋 및 결측치 파악

In [ ]:
#데이터 셋 특징 파악
print(df.info(), "위의 것은 DF의 정보이다.\n\n") #데이터 셋의 정보 확인
print(df.head(3), "위의 것은 DF의 내용이다.\n\n") #데이터 셋 내용 확인
print(df.iloc[:,6:].describe(), "위의 것은 DF의 통계이다\n\n") #데이터프레임의 통계자료 파악(필요없는 칼럼은 제외)
print("GRID_ID의 개수: " + str(len(df["GRID_ID"].unique()))+"\n\n") #격자 종류 : 687 개
print(df['DSP_CNT_IDEX'].value_counts(), "위의 것은 응급출동 회수의 중복값 개수이다\n\n") #중복된 격자의 개수 파악
 #대부분의 격자는 0번의 응급출동이 있고, 최대 14번(1격자)의 응급출동도 있다.

#결측치
print(df.isnull().sum(), "위의 것은 결측치의 개수이다.")
 #isnull()로 결측치 여부를 False, True로 변경한 후 sum()을 통해 합계를 구한다
 #(True면 1, False면 0으로 합계가 결측치의 개수와 동일하다.)
 #각각의 행에서 빠진 값은 없지만, 각각의 날짜의 시간에 해당하는 격자의 개수를 확인하면,
 #몇몇의 격자가 빠져있는 것이 확인가능하다.
  #=>결측치 채우지 않는다 ==> 결측치를 채울 근거가 되는 정보도 없을 뿐더러
  #가장 출동빈도지수가 높은 것을 예측하기 때문에 영향이 적을 것이다.
  #게다가 예측할 날엔 모든 Grid가 해당될 확률이 있으므로 
  #예측할 때 df['GRID_ID'].unique()함수를 통해 모든 그리드를 넣으면, 
  #GRID의 X, Y좌표의 간격에 따라 적절히 예측할 것이다.

# EDA

#### 예측 방법

1. DSP_CNT_IDEX를 예측하는 딥러닝 모델에 필요한 정보는 다음과 같다.
 GRID_X_AXIS, GRID_Y_AXIS, TIME(One-Hot encoding 적용 대상), YMD(M과 D만 분리하여 이용), FP_IDEX, OLD_FP_IDEX 이다.
(GRID_ID의 경우 X축과 Y축의 앞 4글자를 붙인 것으로 그것의 특징을 추출할 수는 있지만, 과적합이 우려된다. 또한 그 특징은 X축과 Y축에서 추출한 특징과 유사하기 때문에 불필요하다. YMD의 Y의 경우 2020으로 일정하기 때문에 딥러닝이라는 자동 특징 추출기에서 추출할 특징이 없다. 마찬가지로 STDG_CD도 일정한 값(같은 지역)을 가지기 때문에 불필요하다. RELIFPLC_DWL_IDEX ~ DISEASE_EXCL_DSP_IDEX는 DSP_CNT_IDEX에서 파생된 즉, DSP_CNT_IDEX의 결과로 생기는 변수로 DSP_CNT_IDEX를 예측하는 문제에서 미리 이 값들을 아는 것은 불가능하며 인과관계가 반대이다.)

2. 1에서 필요한 정보에서 주어지지 않은 것은 FP_IDEX(유동인구 지수)와 OLD_FP_IDEX(노인 유동인구 지수)이다. 

가설: 이 두 값은 7일 후 또는 7일 전과 비슷하다고 예상된다. (주말과 평일의 결과가 다르기 때문이므로 전후일과 비슷하지는 않을 것으로 예상한다.) 두 값의 평균인 이유는 기념일이 포함된 경우 그 영향을 줄이기 위함이다. +14, -14일은 너무 멀다고 판단한다. 그러므로 그래프를 통해 7일마다의 FP_IDEX와 OLD_FP_IDEX의 추세를 파악한 후 적절하다고 여겨지면, 예측할 날짜(월말)의 결측치로 대용한다.
=>상관관계가 있기는 하지만 크지는 않아서 딥러닝으로 예측하는 모델을 만든다.

3. FP_IDEX와 OLD_FP_IDEX가 높으면 DSP_CNT_IDEX의 값이 높을 것이다. 특히 OLD_FP_IDEX는 더 높은 가중치를 가질 것이다. 이외에도 특정 GRID_X_AXIS, GRID_Y_AXIS는 높은 DSP_CNT_IDEX를 가지는데, 이것은 DSP_CNT_IDEX가 높은 인접 GRID에도 DSP_CNT_IDEX가 높아질 가능성을 부여한다.

In [ ]:
#7일마다의 FP_IDEX, OLD_FP_IDEX의 추세

 #모든 시간대를 비교하지는 않고 유동인구가 많을 시각인 18~19시를 비교한다.
print('FP_IDEX, OLD_FP_IDEX의 추세')
eda = df[df['TIME'] == 18] #시간이 18과 같은 값만 df에서 추출하여 eda라는 데이터프레임으로 만든다

#YMD를 그룹으로 하여, 평균을 구한다 => 각 날짜별로 FP_IDEX, OLD_FP_IDEX의 평균이 계산된다.
eda2 = eda.groupby(['YMD']).mean()[['FP_IDEX', 'OLD_FP_IDEX']]
 #eda에서 YMD를 기준으로 평균을 구한 후 평균들 중에서 FP_IDEX와 OLD_FP_IDEX를 추출한다
eda2['YMD'] = eda2.index #인덱스로 설정된 YMD를 eda2의 칼럼으로 만든다
eda2['D'] = eda2['YMD'].astype(str).str[6:].astype(int)
 #YMD을 문자열로 바꾼 후 일자를 추출한 후 정수형으로 변경한다

#그래프
eda2_Jan = eda2[eda2['YMD'] > 20200100] #eda2의 달이 1월부터인 것 추출
eda2_Jan = eda2[eda2['YMD'] < 20200200] #eda2의 달이 2월미만인 것 추출 (결과적으로 1월)

plt.figure(figsize = (20, 5)) #그래프 figure의 크기를 설정한다
FP_graph = sns.barplot(x = 'D', y = 'FP_IDEX', data = eda2)
 #seaborn의 barplot을 이용하는데, x축은 일자, y축은 FP_IDEX의 수치이며 데이터는 eda2를 이용한다

#y값 텍스트로 입력
x = list(range(-1, 29)) #x좌표로 -1에서 28까지 리스트에 담는다(그래프에 약간 왼쪽에 치우친다)
y = np.round(eda2['FP_IDEX'].tolist() ) #y좌표로 FP_IDEX를 반올림하여 이용한다
for index, row in enumerate(x):#x좌표의 값과 인덱스를 enumerate하며 각각의 bar에 y값을 텍스트로 입력한다
  FP_graph.text(row, y[index], y[index] ) #Parameter는 순서대로 x좌표, y좌표, 텍스트로 넣을 값이다
FP_graph.set_title('FP trend by day in January') #plot의 제목을 설정한다

plt.figure(figsize = (20, 5)) #그래프 figure의 크기를 설정한다
OLD_FP_graph = sns.barplot(x = 'D', y = 'OLD_FP_IDEX', data = eda2)
 #seaborn의 barplot을 이용하는데, x축은 일자, y축은 OLD_FP_IDEX의 수치이며 데이터는 eda2를 이용한다
#y값 텍스트로 입력
x = list(range(-1, 29)) #x좌표로 -1에서 28까지 리스트에 담는다(그래프에 약간 왼쪽에 치우친다)
y = np.round(eda2['OLD_FP_IDEX'].tolist() ) #y좌표로 OLD_FP_IDEX를 반올림하여 이용한다
for index, row in enumerate(x):#x좌표의 값과 인덱스를 enumerate하며 각각의 bar에 y값을 텍스트로 입력한다
  OLD_FP_graph.text(row, y[index], y[index] ) #Parameter는 순서대로 x좌표, y좌표, 텍스트로 넣을 값이다 
OLD_FP_graph.set_title('OLD_FP trend by day in January') #plot의 제목을 설정한다
 #20200101: 수요일
 #목요일, 금요일이 그 중에서 가장 높았고, 토일은 목금에 비해 감소하는 추세가 있다.
 #그리고 다시 월요일에 높아지는 추세를 가진다.

#4. Pearson 상관관계를 파악한다
df_hm = df[['GRID_X_AXIS', 'GRID_Y_AXIS', 'TIME', 'YMD',
              'FP_IDEX', 'OLD_FP_IDEX', 'DSP_CNT_IDEX']]
df_hm.corr()

In [ ]:
#YMD, TIME을 기준으로 DSP_CNT_IDEX 파악
print(df.groupby(['YMD', 'TIME']).sum()['DSP_CNT_IDEX'])

In [ ]:
#구급 출동이 많은 시간대를 분석한다
from matplotlib import pyplot as plt

hr = []
DSP_hr = []
for i in range(24):
  DSP_hr.append(sum(df[df['TIME'] == i].groupby(['YMD']).sum()['DSP_CNT_IDEX']))
  hr.append(i)
plt.plot(hr, DSP_hr)
plt.show()

'''
총 날짜는 약 365 - 12 = 363일인데, 최소값은 340으로 거의 매일 구급 출동이 있음을
의미한다. 최대값은 892로 하루에 2번이상 구급출동이 있음을 의미한다.
=> 많이 발생하는 격자 좌표를 분석한다
'''

In [ ]:
#구급 출동의 빈도가 높은 격자 좌표 분석
 #pivot_table() : 컬럼기준으로 데이터 변환
import numpy as np

# 격자 ID 기준으로 DSP_CNT_IDEX 값을 SUM 기준으로 데이터 변환
rect_table = pd.pivot_table(df, values= "DSP_CNT_IDEX" , index=['GRID_ID'], aggfunc=np.sum)

rect_table["GRID_ID"] = rect_table.index.values
print(rect_table, "\n\n")

# 1년간 500번 이상 출동된 격자 ID
rec_500 = np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] >= 500].index)
print(rec_500, "개수:",len(rec_500),"\n\n")

# 1년간 출동횟수가 50이하인 격자 ID
rec_50 = np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] <= 50].index)
print(rec_50, "개수:",len(rec_50),"\n\n")

'''
전체 격자의 개수는 687개이다.
1년간 출동횟수가 50회 이하인 격자의 개수는 641개,
1년간 출동횟수가 500회이상인 격자의 개수는 5개이다
=>500회 이상인 5개의 격자를 시간에 따라 배치한다.
'''

In [ ]:
#DSP_CNT_IDEX가 0인 격자 ID
print(np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] == 0].index))

In [ ]:
#월, 일, 요일 칼럼 추가
import datetime

# 요일 0 ~ 6 정수값 return (월요일 ~ 일요일)
def get_weekday(ymd):
    yyyy = int(ymd/10000)
    mm = int(ymd%10000/100)    
    dd = int(ymd%10000%100)  
    return datetime.date(yyyy,mm,dd).weekday()


In [ ]:
eda = df.copy()
eda["MONTH"] = df["YMD"].apply(lambda x : int(x%10000/100) )
eda["DAY"] = df["YMD"].apply(lambda x : int(x%10000%100) )
eda["WEEKDAY"] = df["YMD"].apply(get_weekday)

#시간과 DSP의 관련성
time_idex = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['TIME'], aggfunc=np.mean)
time_idex["HOUR"] = time_idex.index.values
# seaborn barplot
plt.figure(figsize = (7,4))
sns.barplot(x="HOUR", y="DSP_CNT_IDEX", data=time_idex)

#요일과 DSP의 관련성
weekday_idx = pd.pivot_table(eda, values= "DSP_CNT_IDEX" , index=['WEEKDAY'], aggfunc=np.mean)
weekday_idx["WEEKDAY"] = weekday_idx.index.values
plt.figure(figsize = (7,4))
sns.barplot(x="WEEKDAY", y="DSP_CNT_IDEX", data=weekday_idx)

#월과 DSP의 관련성
month_idx = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['MONTH'], aggfunc=np.mean)
month_idx["MONTH"] = month_idx.index.values
plt.figure(figsize = (7,4))
sns.barplot(x="MONTH", y="DSP_CNT_IDEX", data=month_idx)

In [ ]:
#응급 출동 횟수가 1이상인 데이터의 관련성
print(df.loc[df['DSP_CNT_IDEX']>=1, 'GRID_ID'].shape) #14889
eda = df[df['DSP_CNT_IDEX']>=1]
eda["MONTH"] = eda["YMD"].apply(lambda x : int(x%10000/100) )
eda["DAY"] = eda["YMD"].apply(lambda x : int(x%10000%100) )
eda["WEEKDAY"] = eda["YMD"].apply(get_weekday)

#시간과 DSP의 관련성
time_idex = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['TIME'], aggfunc=np.mean)
time_idex["HOUR"] = time_idex.index.values
# seaborn barplot
plt.figure(figsize = (7,4))
sns.barplot(x="HOUR", y="DSP_CNT_IDEX", data=time_idex)

#요일과 DSP의 관련성
weekday_idx = pd.pivot_table(eda, values= "DSP_CNT_IDEX" , index=['WEEKDAY'], aggfunc=np.mean)
weekday_idx["WEEKDAY"] = weekday_idx.index.values
plt.figure(figsize = (7,4))
sns.barplot(x="WEEKDAY", y="DSP_CNT_IDEX", data=weekday_idx)

#월과 DSP의 관련성
month_idx = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['MONTH'], aggfunc=np.mean)
month_idx["MONTH"] = month_idx.index.values
plt.figure(figsize = (7,4))
sns.barplot(x="MONTH", y="DSP_CNT_IDEX", data=month_idx)

#=>일단 응급출동이 발생한다면, 시간, 날짜와 관련이 크지는 않다


In [ ]:
#응급 출동 횟수가 0인 데이터의 관련성
print(df.loc[df['DSP_CNT_IDEX']==0, 'GRID_ID'].shape) #14889
eda = df[df['DSP_CNT_IDEX']==0]
eda["MONTH"] = eda["YMD"].apply(lambda x : int(x%10000/100) )
eda["DAY"] = eda["YMD"].apply(lambda x : int(x%10000%100) )
eda["WEEKDAY"] = eda["YMD"].apply(get_weekday)

#시간과 DSP의 관련성
time_idex = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['TIME'], aggfunc=np.mean)
time_idex["HOUR"] = time_idex.index.values
hr = []
for i in range(24):
  hr.append(eda[eda['TIME']==i].shape[0])
# seaborn barplot
plt.figure(figsize = (7,4))
sns.barplot(x="HOUR", y=hr, data=time_idex)
#=>10시~18시에 응급 출동이 0일 가능성이 높다
#df.shape[0] / 24 = 73712로 0시 ~ 6시는  2일에 한 번 응급출동이 있을 예정이므로
#모두 공백처리하면 0시 ~ 6시의 약 50%는 예측 성공이다.(6시는 6-7을 의미한다)

#요일과 DSP의 관련성
weekday_idx = pd.pivot_table(eda, values= "DSP_CNT_IDEX" , index=['WEEKDAY'], aggfunc=np.mean)
weekday_idx["WEEKDAY"] = weekday_idx.index.values #0: 월...6: 일
day = []
for i in range(7):
  day.append(eda[eda['WEEKDAY']==i].shape[0])
#seaborn barplot
plt.figure(figsize = (7,4))
sns.barplot(x="WEEKDAY", y=day, data=weekday_idx)
#=>요일별로 유의미한 특징은 없다

#월과 DSP의 관련성
month_idx = pd.pivot_table(eda, values= ["DSP_CNT_IDEX"] , index=['MONTH'], aggfunc=np.mean)
month_idx["MONTH"] = month_idx.index.values
m = []
for i in range(12):
  m.append(eda[eda['MONTH']==i].shape[0])
#seaborn barplot
plt.figure(figsize = (7,4))
sns.barplot(x="MONTH", y=m, data=month_idx)

#=>1월에 응급출동이 항상 있었다는 점을 제외하면, 유의미한 특징은 없다
#(1월은 응급출동을 0으로 예측한다)

In [ ]:
#예측하려는 7시~23시 사이에 응급출동 횟수가 많은 격자 ID 파악
k = df[df['TIME']>=7]

# 격자 ID 기준으로 DSP_CNT_IDEX 값을 SUM 기준으로 데이터 변환
rect_table = pd.pivot_table(k, values= "DSP_CNT_IDEX" , index=['GRID_ID'], aggfunc=np.sum)
rect_table["GRID_ID"] = rect_table.index.values

# 1년간 250번(해당되는 시간 * 날짜) 이상 출동된 격자 ID
rec_250 = np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] >= 250].index)
print(rec_250, "개수:",len(rec_250),"\n\n")

'''
좌표 시각화
39445294
39445284  39545284 
39445274  39545274  39645274
39445264  39545264  39645264
          39545254
          39545244  39645244

가장 많은 격자 ID는 39445274, 39545274, 39545264, 39645264, 39545254, 39645244 가 있다.
'''

# 학습

## 학습1
- 달과 일(YMD에서 추출), GRID_X_AXIS, GRID_Y_AXIS, 시간(원핫인코딩)으로 FP_IDEX와 OLD_FP_IDEX를 예측하는 모델을 만들어서 월말의 FP_IDEX와 OLD_FP_IDEX를 예측하는 모델을 생성한다.
- 입력 변수 = 달 / 일 / 시간 / 격자 X / 격자 Y
- input = [MONTH, DAY, GRID_X_AXIS, GRID_Y_AXIS]


## 학습2
- 학습1에서 구한 FP_IDEX, OLD_FP_IDEX와 GRID_X_AXIS, GRID_Y_AXIS, 달, 일, TIME(원핫인코딩), 격자 ID를 독립변수로 하여 DSP 여부를 예측한다. (그 이후에 가장 높은 DSP 확률을 가진 GRID_ID를 답안으로 한다.)
- 입력 변수 = 유동인구/ 노인유동인구/ 격자 X/ 격자 Y/ 달/ 일/ 시간/ 격자ID
- input = [FP_IDEX, OLD_FP_IDEX, GRID_X_AXIS, GRID_Y_AXIS, MONTH, DAY, TIME, GRID_ID]

## 학습1 - 전처리

In [ ]:
#전처리
df1 = df.copy() #전처리 전 데이터 프레임을 복제한다.

#DSP_CNT_IDEX가 500이하인 값들은 제거
# 격자 ID 기준으로 DSP_CNT_IDEX 값을 SUM 기준으로 데이터 변환
rect_table = pd.pivot_table(df, values= "DSP_CNT_IDEX" , index=['GRID_ID'], aggfunc=np.sum)
rect_table["GRID_ID"] = rect_table.index.values
rec_all = df['GRID_ID'].unique()
rec_500 = np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] >= 500].index)
rec_diff = set(rec_all) - set(rec_500)
for id in rec_diff:
  df1.drop(df1[df1['GRID_ID']==id].index, inplace=True)
#(500이상인 x개의 GRID ID를 시간대에 배치하는 문제로 전환)

#1월은 응급출동회수를 0으로 예측하므로 예측에 포함시키지 않는다 
df1.drop(df1[ (df1['YMD'] / 100).astype(int) ==202001].index, inplace = True)

#STDG_CD(법정동코드) 제거
 #모든 데이터에 대해 동일하여 예측하는 데 도움이 되지 않기 때문에
 #과적합 방지, 그리고 데이터 학습 속도 향상을 위해 제거한다
df1.drop('STDG_CD', axis=1, inplace=True) #STDG_CD라는 칼럼을 제거하고 df1에 바로 반영한다

#GRID_ID 제거
 #격자 X축 좌표와 격자 Y축 좌표는 각각의 변수끼리 1000씩 변하기 때문에 
 #딥러닝 학습에서 응급 출동을 파악할 때 인근지역이라는 특징을 추출할 것이다. 
 #하지만 격자 ID는 격자 X축 좌표와 격자 Y축 좌표의 앞 4글자를 따서 이어붙인 것으로 
 #딥러닝이 특징을 파악할 수는 있겠지만, 그 과정에서 오버피팅이 발생할 확률이 매우 높다. 
 #게다가 격자 ID에서 추출하고 싶은 특징은 격자 X축 좌표와 격자 Y축 좌표에서 
 #이미 추출한 특징과 중복되므로 학습 전 제거하는 것이 바람직하다.
df1.drop('GRID_ID', axis=1, inplace=True) #GRID_ID라는 칼럼을 제거하고 df1에 바로 반영한다

#RELIFPLC_DWL_IDEX ~ DISEASE_EXCL_DSP_IDEX 제거
 #테이블정의서 파일의 구급처주거지수~질병외출동지수인 8개의 변수는 
 #예측하고자 하는 변수인 FP_IDEX와 OLD_FP_IDEX 파생된 것으로 제거한다.
df1.drop('RELIFPLC_DWL_IDEX', axis=1, inplace=True) #RELIFPLC_DWL_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('RELIFPLC_ROAD_IDEX', axis=1, inplace=True) #RELIFPLC_ROAD_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('RELIFPLC_INDUST_IDEX', axis=1, inplace=True) #RELIFPLC_INDUST_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('RELIFPLC_NTR_IDEX', axis=1, inplace=True) #RELIFPLC_NTR_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('RELIFPLC_ETC_IDEX', axis=1, inplace=True) #RELIFPLC_ETC_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('DISEASE_DSP_IDEX', axis=1, inplace=True) #DISEASE_DSP_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('DISEASE_EXCL_DSP_IDEX', axis=1, inplace=True) #DISEASE_EXCL_DSP_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다
df1.drop('DSP_CNT_IDEX', axis = 1, inplace = True) #DSP_CNT_IDEX라는 칼럼을 제거하고 df1에 바로 반영한다.

#YMD를 YEAR, MONTH, DAY로 분리하고 YEAR을 제거한다
 #YMD를 YEAR, MONTH, DAY로 분리하여 특징을 더 잘 파악할 수 있도록 한다.
df1['YMD'] = df1['YMD'].astype(str) #YMD의 타입을 문자열로 변경한다
df1['YEAR'] = df1['YMD'].str[0:4] #8자리 문자열의 0,1,2,3번째를 추출한다
df1['MONTH'] = df1['YMD'].str[4:6] #8자리 문자열의 4, 5번째를 추출한다
df1['DAY'] = df1['YMD'].str[6:8] #8자리 문자열의 6, 7번째를 추출한다
df1.drop('YMD', axis=1, inplace=True) #YMD는 분리하였으므로 제거한다
df1.drop('YEAR', axis=1, inplace=True) #년은 2020으로 일정하므로 특징이 없어서 제거한다
df1['MONTH'] = df1['MONTH'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다 
df1['DAY'] = df1['DAY'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다

#TIME은 범주형 데이터이므로 One-Hot encoding을 적용한다
 #범주형 데이터가 숫자값으로 되어 학습을 오도하는 것을 방지한다
 #YMD와 TIME 모두 One-Hot encoding을 적용하면 데이터가 너무 커지기 때문에
 #둘 중 Pearson 상관계수가 큰 feature인 TIME만 One-Hot encoding을 적용한다
df1['TIME'] = df1['TIME'].astype(str) #TIME 열을 문자열 타입으로 변경한다
ohe_time = pd.get_dummies(df1[['TIME']]) #One-Hot encoding을 적용한다
df1.drop(['TIME'], axis=1, inplace=True) #이용하지 않는 칼럼(TIME) 제거

#6. 표준화(Standardization)
 #이상치에 강하며 데이터 학습 시간을 단축시킨다.
 #스케일이 큰 Feature의 영향을 줄여 오버피팅 방지에 도움준다
 #표준화는 수치형 변수(One-Hot encoding하지 않은)에만 수행하기 때문에 
 #원-핫인코딩한 칼럼을 미리 제거한다
fp = df1[['FP_IDEX', 'OLD_FP_IDEX']].copy() #y값이므로 표준화에서 제외하기 위해 복사한다
                              #(shallow copy는 값이 함께 변하므로 deep copy를 이용한다)
df1.drop(['FP_IDEX', 'OLD_FP_IDEX'], axis=1, inplace=True) #FP_IDEX, OLD_FP_IDEX 칼럼 제거
df1_res = (df1 - df1.mean()) / df1.std() #표준화 적용
df1_res = pd.concat([df1_res, ohe_time, fp], axis=1)

#변경된 타입 확인
print(df1_res.dtypes, "위의 것은 DF의 타입이다.")

## 학습1 - 딥러닝 학습

In [ ]:
print(df1_res)

In [ ]:
#독립변수와 종속변수를 분리
#독립 칼럼을 데이터프레임에서 추출한다
y1 = df1_res[['FP_IDEX', 'OLD_FP_IDEX']].copy()
x1 = df1_res.drop(['FP_IDEX', 'OLD_FP_IDEX'], axis=1, inplace = False)
#train, test 분리
from sklearn.model_selection import train_test_split #분리하는 라이브러리 Import
trainX1, testX1, trainY1, testY1 = train_test_split(x1, y1, test_size = 0.3, random_state = 50) #test, train 데이터 셋을 각각 30%, 70%로 분리

In [ ]:
#trainX1의 모양 확인 (딥러닝 학습 시 인풋 모양 결정에 필요)
print(trainX1.shape) #(데이터 갯수, 칼럼 갯수)

#trainY1의 모양 확인 (딥러닝 학습 시 아웃풋 모양 결정에 필요)
print(trainY1.shape) #(데이터 갯수, 칼럼 갯수)

In [ ]:
def residual_block(X):
      #ResNet 구조를 이용한다 (배치 Normalization과 Activation을 쌓는다)
  H = tf.keras.layers.BatchNormalization()(X)
  H = tf.keras.layers.Activation(keras.activations.gelu)(H)
  
  H = tf.keras.layers.BatchNormalization()(H)
  H = tf.keras.layers.Activation('gelu')(H)
  
  #가중치를 잘 전달하기 위해서 마지막엔 activation 안한다
  H = tf.keras.layers.Add()([X, H])

  return H

In [ ]:
# 모델

#He Normalization
initializer = tf.keras.initializers.HeNormal(seed=None)
#Relu계열에서 잘 작동하는 Normalization이용하여 가중치를 초기화한다
#(Xavier는 Relu계열에서 잘 작동하지 않는다)

X = tf.keras.layers.Input(shape=[28])
H = tf.keras.layers.BatchNormalization()(X)

H = tf.keras.layers.Dense(32, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.1)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H) #L1 L2규제
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(2):
  H = residual_block(H)
H = tf.keras.layers.Dense(64, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.1)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H) #L1 L2규제
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(2):
  H = residual_block(H)
H = tf.keras.layers.Dense(128, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(4):
  H = residual_block(H)
H = tf.keras.layers.Dense(256, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(4):
  H = residual_block(H)
H = tf.keras.layers.Dense(512, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(6):
  H = residual_block(H)
H = tf.keras.layers.Dense(1024, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.3)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

H = tf.keras.layers.Dense(128, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다
H = tf.keras.layers.ActivityRegularization(l1 = 0.001, l2 = 0.01)(H)

Y = tf.keras.layers.Dense(2)(H)

model1 = tf.keras.models.Model(X, Y)

keras.utils.plot_model(model1) #모델 시각화

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Nadam
import tensorflow as tf

#local minimum에서 탈출할 수 있도록 cosine decay restarts 이용
batch_size = 32 #배치 사이즈
#initial_learning_rate = 0.1*batch_size/256 #초기 학습률
initial_learning_rate = 0.8
first_decay_steps = batch_size * 20 #20 Epochs 동안 진행
lr_rdecayed_fn = tf.keras.experimental.CosineDecayRestarts(
    initial_learning_rate, first_decay_steps, t_mul=batch_size*20, m_mul=0.5, alpha=0.0,
    name='RDecay'
) #Local Minimum에서 탈출할 수 있도록 Cosine Decay Restarts 이용
#Cosine 형태로 learning rate가 절반으로 감소하는데, 20 epochs마다 reset된다

#Early stopping
callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 50)
mc = keras.callbacks.ModelCheckpoint('best_model1.h5', monitor='val_loss', mode='min', save_best_only=True)

#SGD
model1.compile(optimizer= SGD(learning_rate=lr_rdecayed_fn, decay=1E-6, momentum=0.9,
                              nesterov=True, clipnorm=0.001),
              loss= tf.losses.mean_squared_error,
              metrics= ['mean_absolute_error']) # keras.metrics.mean_squared_error


#학습
hist1 = model1.fit(trainX1, trainY1, epochs=200,
                     batch_size = 32, shuffle=True, validation_split=0.25,
                     callbacks=[callbacks, mc], verbose=1)


In [ ]:
#모델 저장
from keras.models import load_model
model1.save('deep_learning_model1.h5')
model1_best = keras.models.load_model('best_model1.h5')

In [ ]:
#모델 평가 및 시각화
model1.evaluate(testX1, testY1)
model1_best.evaluate(testX1, testY1)
print(hist1.history)
print(len(hist1.history['loss']))
print(hist1.history['mean_absolute_error'])

#훈련 과정 시각화 (MAE)
plt.plot(hist1.history["mean_absolute_error"])
plt.plot(hist1.history['val_mean_absolute_error'])
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

#훈련 과정 시각화 (손실)
plt.plot(hist1.history['loss'])
plt.plot(hist1.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model1.summary()

In [ ]:
#예측값과 실제값을 비교한다
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

y1 = testY1.values #딕셔너리의 value 값을 변수에 저장한다
y_pred1 = model1.predict(testX1) #모델이 예측 결과가 반환된다.
y1 = pd.DataFrame(data = y1, columns = ['FP_IDEX', 'OLD_FP_IDEX'])
 #y1을 이차원 넘파이 어레이에서 데이터프레임으로 변경한다.
y_pred1 = pd.DataFrame(data = y_pred1, columns = ['Pred_FP_IDEX', 'Pred_OLD_FP_IDEX'])
 #y_pred1을 이차원 넘파이 어레이에서 데이터프레임으로 변경한다.

#y1와 y_pred1를 비교한다(y = x 축 상에 놓여있을 수록 정확히 예측한 것이다.)
plt.scatter(y1['FP_IDEX'], y_pred1['Pred_FP_IDEX'], s = 1)
 #x축은 y, y축은 y_pred로 scatter plot을 이용하여 y=x그래프에 얼마나 가까운지 확인한다
x_check=list(range(0, 12000)) #y=x그래프의 x범위
y_check=list(range(0, 12000)) #y=x그래프의 y범위
plt.plot(x_check, y_check, color='red') #성능 확인을 위한 y=x 그래프를 그린다
plt.title("Comparing y with y_hat(FP_IDEX)",fontsize=15) #제목 설정
plt.xlabel("y",fontsize=13) #x축 label 설정
plt.ylabel("y_hat",fontsize=13) #y축 label 설정
plt.show() #그래프 보기

plt.scatter(y1['OLD_FP_IDEX'], y_pred1['Pred_OLD_FP_IDEX'], s = 1)
 #x축은 y, y축은 y_pred로 scatter plot을 이용하여 y=x그래프에 얼마나 가까운지 확인한다
x_check=list(range(0, 2000)) #y=x그래프의 x범위
y_check=list(range(0, 2000)) #y=x그래프의 y범위
plt.plot(x_check, y_check, color='red') #성능 확인을 위한 y=x 그래프를 그린다
plt.title("Comparing y with y_hat(OLD_FP_IDEX)",fontsize=15) #제목 설정
plt.xlabel("y",fontsize=13) #x축 label 설정
plt.ylabel("y_hat",fontsize=13) #y축 label 설정
plt.show() #그래프 보기

In [ ]:
#모델 평가하기 - RMSE
from sklearn.metrics import mean_squared_error
import numpy as np
def RMSE(testY, y_hat):
  return np.sqrt(mean_squared_error(testY, y_hat))
print(f'RMSE: {RMSE(testY1, y_pred1)}')
 #RMSE는 정규화 이후에 평가한 것으로 평균과 분산 등을 고려하여 이해할 필요가 있다

## 학습2 - 전처리

In [ ]:
#응급출동 회수가 1이상인 행의 개수
print(df.loc[df['DSP_CNT_IDEX']>=1, 'GRID_ID'].shape) #14889

In [ ]:
df2 = df.copy() #전처리 전 데이터 프레임을 복제한다.

#DSP_CNT_IDEX가 500이하인 값들은 제거
# 격자 ID 기준으로 DSP_CNT_IDEX 값을 SUM 기준으로 데이터 변환
rect_table = pd.pivot_table(df, values= "DSP_CNT_IDEX" , index=['GRID_ID'], aggfunc=np.sum)
rect_table["GRID_ID"] = rect_table.index.values
rec_all = df['GRID_ID'].unique()
rec_500 = np.unique(rect_table["DSP_CNT_IDEX"][rect_table["DSP_CNT_IDEX"] >= 500].index)
rec_diff = set(rec_all) - set(rec_500)
for id in rec_diff:
  df2.drop(df2[df2['GRID_ID']==id].index, inplace=True)
#(500이상인 x개의 GRID ID를 시간대에 배치하는 문제로 전환)

#1월은 응급출동회수를 0으로 예측하므로 예측에 포함시키지 않는다 
df2.drop(df2[ (df2['YMD'] / 100).astype(int) ==202001].index, inplace = True)

#STDG_CD(법정동코드) 제거
 #모든 데이터에 대해 동일하여 예측하는 데 도움이 되지 않기 때문에
 #과적합 방지, 그리고 데이터 학습 속도 향상을 위해 제거한다
df2.drop('STDG_CD', axis=1, inplace=True) #STDG_CD라는 칼럼을 제거하고 df2에 바로 반영한다

#DSP_CNT_IDEX를 제외한 RELIFPLC_DWL_IDEX ~ DISEASE_EXCL_DSP_IDEX 제거
 #테이블정의서 파일의 (출동빈도지수를 제외한) 구급처주거지수~질병외출동지수인 7개의 변수는 
 #예측하고자 하는 변수인 출동빈도지수에서 파생된 것으로 
 #월말의 유동인구지수..등을 모르므로 이 값을 예측할 때 파라미터로 이용하는 것은 바람직하지 못하다.
df2.drop('RELIFPLC_DWL_IDEX', axis=1, inplace=True) #RELIFPLC_DWL_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('RELIFPLC_ROAD_IDEX', axis=1, inplace=True) #RELIFPLC_ROAD_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('RELIFPLC_INDUST_IDEX', axis=1, inplace=True) #RELIFPLC_INDUST_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('RELIFPLC_NTR_IDEX', axis=1, inplace=True) #RELIFPLC_NTR_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('RELIFPLC_ETC_IDEX', axis=1, inplace=True) #RELIFPLC_ETC_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('DISEASE_DSP_IDEX', axis=1, inplace=True) #DISEASE_DSP_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다
df2.drop('DISEASE_EXCL_DSP_IDEX', axis=1, inplace=True) #DISEASE_EXCL_DSP_IDEX라는 칼럼을 제거하고 df2에 바로 반영한다

#TIME이 06시 이하인 것을 제거
df2 = df2[df2['TIME']>6]

#YMD를 YEAR, MONTH, DAY로 분리하고 YEAR을 제거한다
 #YMD를 YEAR, MONTH, DAY로 분리하여 특징을 더 잘 파악할 수 있도록 한다.
df2['YMD'] = df2['YMD'].astype(str) #YMD의 타입을 문자열로 변경한다
df2['YEAR'] = df2['YMD'].str[0:4] #8자리 문자열의 0,1,2,3번째를 추출한다
df2['MONTH'] = df2['YMD'].str[4:6] #8자리 문자열의 4, 5번째를 추출한다
df2['DAY'] = df2['YMD'].str[6:8] #8자리 문자열의 6, 7번째를 추출한다
#df2.drop('YMD', axis=1, inplace=True) #YMD는 분리하였으므로 제거한다
df2.drop('YEAR', axis=1, inplace=True) #년은 2020으로 일정하므로 특징이 없어서 제거한다
df2['MONTH'] = df2['MONTH'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다 
df2['DAY'] = df2['DAY'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다

#TIME은 범주형 데이터이므로 One-Hot encoding을 적용한다
 #범주형 데이터가 숫자값으로 되어 학습을 오도하는 것을 방지한다
 #YMD와 TIME 모두 One-Hot encoding을 적용하면 데이터가 너무 커지기 때문에
 #둘 중 Pearson 상관계수가 큰 feature인 TIME만 One-Hot encoding을 적용한다
df2['TIME'] = df2['TIME'].astype(str) #TIME 열을 문자열 타입으로 변경한다
df2['DATE'] = df2['YMD'].astype(str) + df2['TIME'] #YMD오 TIME을 이은 DATE 칼럼 생성 (구분하는 데 이용)
ohe_time = pd.get_dummies(df2[['TIME']]) #One-Hot encoding을 적용한다
df2.drop(['DATE', 'YMD'], axis=1, inplace=True)
df2.drop(['TIME'], axis=1, inplace=True) #불필요한 칼럼 제거 

#DSP_CNT_IDEX를 0개, 1개 이상인 2개의 범주로 나누어 One-Hot encoding을 적용한다
df2.loc[df2['DSP_CNT_IDEX'] == 0, 'DSP'] = 'NO' #DSP_CNT가 0인 행을 DSP에 NO로 입력
df2.loc[df2['DSP_CNT_IDEX'] > 0, 'DSP'] = 'YES' #DSP_CNT가 1인 행을 DSP에 YES으로 입력
ohe_DSP = pd.get_dummies(df2[['DSP']]) #One-Hot encoding 적용

#7. GRID_ID One-Hot encoding 적용
df2['GRID_ID'] = df2['GRID_ID'].astype(str)
ohe_grid = pd.get_dummies(df2[['GRID_ID']])
df2.drop(['GRID_ID'], axis=1, inplace=True)

#8. 정규화(Min-Max Normalization)
 #이상치에 약하고 데이터 학습 시간을 단축시킨다.
 #스케일이 큰 Feature의 영향을 줄여 오버피팅 방지에 도움준다
 #정규화는 수치형 변수(One-Hot encoding하지 않은)에만 수행하기 때문에 
 #원-핫인코딩한 칼럼을 미리 제거한다
df2.drop(['DSP_CNT_IDEX'], axis=1, inplace=True) #불필요한 칼럼 제거
df2.drop(['DSP'], axis=1, inplace=True) #불필요한 칼럼 제거

df2_res = (df2 - df2.mean()) / (df2.std()) #정규화 적용 (1, 0사이라서 loss커질 수도있음)
df2_res = pd.concat([df2_res, ohe_time, ohe_grid, ohe_DSP], axis=1) #정규화된 수치형 데이터와 범주형 데이터 결합.(입력변수만 정규화되도록)

#변경된 타입 확인
print(df2_res.dtypes, "위의 것은 DF의 타입이다.")

## 학습2 - 딥러닝 학습

In [ ]:
print(df2_res)

In [ ]:
#Nan이나 Inf있는지 확인
print(df2_res.isnull().any())

In [ ]:
print(df2_res.columns) #칼럼 출력

In [ ]:
#독립변수와 종속변수를 분리
y2 = df2_res[['DSP_YES', 'DSP_NO']] #종속 칼럼을 데이터프레임에서 추출한다
x2 = df2_res.drop(['DSP_YES', 'DSP_NO'], axis=1) #독립 칼럼을 데이터프레임에서 추출한다

#train, test 분리
from sklearn.model_selection import train_test_split #분리하는 라이브러리 Import
trainX2, testX2, trainY2, testY2 = train_test_split(x2, y2, test_size = 0.3, random_state = 50) #test, train 데이터 셋을 각각 30%, 70%로 분리

#trainX의 모양 확인 (딥러닝 학습 시 인풋 모양 결정에 필요)
print(trainX2.shape) #(데이터 갯수, 칼럼 갯수)

#trainY의 모양 확인 (딥러닝 학습 시 아웃풋 모양 결정에 필요)
print(trainY2.shape) #(데이터 갯수, 칼럼 갯수)

In [ ]:
def residual_block(X):
      #ResNet 구조를 이용한다 (배치 Normalization과 Activation을 쌓는다)
  H = tf.keras.layers.Activation(keras.activations.gelu)(X)
  H = tf.keras.layers.BatchNormalization()(H)
  #Pre-Activation 구조를 통해 가중치 최대한 전달한다
  
  H = tf.keras.layers.Activation('gelu')(H)
  H = tf.keras.layers.BatchNormalization()(H)
  
  #가중치를 잘 전달하기 위해서 마지막엔 activation 안한다
  H = tf.keras.layers.Add()([X, H])

  return H

In [ ]:
# 모델

#He Normalization
initializer = tf.keras.initializers.HeNormal(seed=None)
#Xavier는 Relu계열에서 비효율적이기 때문에 He 초기화를 이용한다

X = tf.keras.layers.Input(shape=[28])
H = tf.keras.layers.BatchNormalization()(X)

H = tf.keras.layers.Dense(32, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(2):
  H = residual_block(H)
H = tf.keras.layers.Dense(64, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(4):
  H = residual_block(H)
H = tf.keras.layers.Dense(128, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.3)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(8):
  H = residual_block(H)
H = tf.keras.layers.Dense(256, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.3)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(16):
  H = residual_block(H)
H = tf.keras.layers.Dense(512, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.4)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

for i in range(16):
  H = residual_block(H)
H = tf.keras.layers.Dense(1024, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.4)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

H = tf.keras.layers.Dense(256, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.3)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

H = tf.keras.layers.Dense(64, kernel_initializer=initializer)(H)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation('gelu')(H)
H = tf.keras.layers.Dropout(0.2)(H) # 드롭아웃 추가.
H = tf.keras.layers.ActivityRegularization(l1 = 0, l2 = 0.1)(H)
#gelu 함수를 통해 가중치가 잘 전달되고, 빠른 속도로 동작하게 한다

Y = tf.keras.layers.Dense(2, activation='softmax')(H)
#softmax 함수를 통해 결과가 확률이 되도록 한다

model2 = tf.keras.models.Model(X, Y)

keras.utils.plot_model(model2) #모델 시각화

In [ ]:
pip install tensorflow_addons #텐서 플로우 에드온 다운로드

In [ ]:
from tensorflow.keras.optimizers import Adam, SGD, Nadam
import tensorflow as tf
import tensorflow_addons as tfa


#local minimum에서 탈출할 수 있도록 cosine decay restarts 이용
batch_size = 32 #배치 사이즈
#initial_learning_rate = 0.1*batch_size/256 #초기 학습률
initial_learning_rate = 0.01
first_decay_steps = batch_size * 20 #10 Epochs 동안 진행
lr_rdecayed_fn = tf.keras.experimental.CosineDecayRestarts(
    initial_learning_rate, first_decay_steps, t_mul=batch_size*20, m_mul=0.5, alpha=0.0,
    name='RDecay'
)
#Local Minimum에서 탈출할 수 있도록 Cosine Decay Restarts 이용

#Early stopping
callbacks = keras.callbacks.EarlyStopping(monitor='val_loss', patience = 50)
mc = keras.callbacks.ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', save_best_only=True)

#SGD
model2.compile(optimizer= SGD(learning_rate=lr_rdecayed_fn, decay=1E-6, momentum=0.9,
                              nesterov=True, clipnorm=0.001),
              loss= tfa.losses.SigmoidFocalCrossEntropy(alpha=0.25, gamma=2,
                                                        reduction=tf.keras.losses.Reduction.AUTO),
              metrics= tf.keras.metrics.Precision(thresholds = 0.099))

hist2 = model2.fit(trainX2, trainY2, epochs=200,
                     batch_size = 32, shuffle=True, validation_split=0.25,
                     callbacks=[callbacks, mc], verbose=1)


In [ ]:
#모델 저장
from keras.models import load_model
model2.save('deep_learning_model2.h5')
model2_best = keras.models.load_model('best_model2.h5')

In [ ]:
#모델 학습 과정 표시하기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist2.history['loss'], 'y', label='train loss')
loss_ax.plot(hist2.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist2.history['precision'], 'b', label='train acc')
acc_ax.plot(hist2.history['val_precision'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
pip install tensorflow_addons #텐서플로우 에드온 다운로드

In [ ]:
#모델 불러오기
from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
import tensorflow as tf
model2 = load_model('best_model2.h5', 
                    custom_objects={'loss':tfa.losses.SigmoidFocalCrossEntropy(alpha=0.25, gamma=2,
                                                        reduction=tf.keras.losses.Reduction.AUTO)})

In [ ]:
y2 = testY2.values #딕셔너리의 value 값을 변수에 저장한다
y_pred2 = model2.predict(testX2) #모델이 예측 결과가 반환된다.

In [ ]:
import pandas as pd
y2_df = testY2
y_pred2_df = pd.DataFrame(data = y_pred2, columns = ['DSP_YES', 'DSP_NO'])
y2_df.reset_index(drop=True, inplace=True)
y_pred2_df.reset_index(drop=True, inplace=True)

In [ ]:
y_pred2_df[y_pred2_df['DSP_YES']>0.31443]

In [ ]:
print(y2_df[4:5])
print(y2_df[21:22])
print(y2_df[37:38])
print(y2_df[42:43])
print(y2_df[64:65])
print(y2_df[8202:8203])
print(y2_df[8219:8220])
print(y2_df[8220:8221])
print(y2_df[8223:8224])
print(y2_df[8229:8230])

In [ ]:
y2_df[y2_df['DSP_YES']>0]

# 답안 생성

- DSP_YES 가 높은 순서대로 각 날짜에 격자ID 삽입한다.

In [ ]:
pip install tensorflow_addons #텐서플로우 에드온 다운로드

In [ ]:
#구글 드라이브 마운트 (구글 드라이브에 제공된 데이터 셋 업로드 후 이용)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import keras

from tensorflow.keras.models import load_model
import tensorflow_addons as tfa
import tensorflow as tf

#데이터 셋 불러오기
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/소방안전AI예측/02.격자단위_구급출동_dataset.csv', encoding = 'euc-kr')

#모델 불러오기
model1 = keras.models.load_model('deep_learning_model1.h5')
model2 = keras.models.load_model('deep_learning_model2.h5',
                                 custom_objects={'loss':tfa.losses.SigmoidFocalCrossEntropy(alpha=0.25, gamma=2,
                                                        reduction=tf.keras.losses.Reduction.AUTO)})

In [ ]:
#답안 예측을 위한 독립 변수 생성
import pandas as pd
results = []

#날짜(1월 제외)
Pred_Dates = [20200228, 20200331, 20200430, 20200531, 20200630,
             20200731, 20200831, 20200930, 20201031, 20201130, 20201231]
# 시간대 (0 ~ 23)
Pred_Hours = [i for i in range(0, 24)]
# 격자들
GRID_IDs = [39445274, 39545254, 39545274, 39645244, 39645264]

#빈 데이터 프레임
Pred_df = pd.DataFrame(index = range(0, len(Pred_Dates)*len(Pred_Hours)*len(GRID_IDs)), 
                       columns=['YMD', 'TIME', 'GRID_X_AXIS', 'GRID_Y_AXIS', 'GRID_ID'])

i = 0
for date in Pred_Dates:
  for hour in Pred_Hours:
    for grid_id in GRID_IDs:
      # 격자 ID 로부터 격자 X 좌표 / 격자 Y 좌표 계산
      Pred_df['GRID_ID'][i] = grid_id
      Pred_df['GRID_X_AXIS'][i] = int(grid_id/10000)*100 + 75
      Pred_df['GRID_Y_AXIS'][i] = int(grid_id%10000)*100 + 75
      Pred_df['YMD'][i] = date
      Pred_df['TIME'][i] = hour
      i = i+1

Pred_df = Pred_df.astype(int)

In [ ]:
Pred_df.dtypes

In [ ]:
Pred_df.to_csv('정답.csv')

In [ ]:
#model1을 이용해서 FP_IDEX와 OLD_FP_IDEX 칼럼을 생성한다
#전처리
Pred_df_c1 = Pred_df.copy() #전처리 전 데이터 프레임을 복제한다.

Pred_df_c1.drop('GRID_ID', axis=1, inplace=True)

#YMD를 YEAR, MONTH, DAY로 분리하고 YEAR을 제거한다
 #YMD를 YEAR, MONTH, DAY로 분리하여 특징을 더 잘 파악할 수 있도록 한다.
Pred_df_c1['YMD'] = Pred_df_c1['YMD'].astype(str) #YMD의 타입을 문자열로 변경한다
Pred_df_c1['YEAR'] = Pred_df_c1['YMD'].str[0:4] #8자리 문자열의 0,1,2,3번째를 추출한다
Pred_df_c1['MONTH'] = Pred_df_c1['YMD'].str[4:6] #8자리 문자열의 4, 5번째를 추출한다
Pred_df_c1['DAY'] = Pred_df_c1['YMD'].str[6:8] #8자리 문자열의 6, 7번째를 추출한다
Pred_df_c1.drop('YMD', axis=1, inplace=True) #YMD는 분리하였으므로 제거한다
Pred_df_c1.drop('YEAR', axis=1, inplace=True) #년은 2020으로 일정하므로 특징이 없어서 제거한다
Pred_df_c1['MONTH'] = Pred_df_c1['MONTH'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다 
Pred_df_c1['DAY'] = Pred_df_c1['DAY'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다

#TIME은 범주형 데이터이므로 One-Hot encoding을 적용한다
 #범주형 데이터가 숫자값으로 되어 학습을 오도하는 것을 방지한다
 #YMD와 TIME 모두 One-Hot encoding을 적용하면 데이터가 너무 커지기 때문에
 #둘 중 Pearson 상관계수가 큰 feature인 TIME만 One-Hot encoding을 적용한다
Pred_df_c1['TIME'] = Pred_df_c1['TIME'].astype(str) #TIME 열을 문자열 타입으로 변경한다
ohe_time = pd.get_dummies(Pred_df_c1[['TIME']]) #One-Hot encoding을 적용한다
Pred_df_c1.drop(['TIME'], axis=1, inplace=True) #표준화 적용하지 않는 칼럼(TIME) 제거

#표준화(Standardization)
 #이상치에 강하며 데이터 학습 시간을 단축시킨다.
 #스케일이 큰 Feature의 영향을 줄여 오버피팅 방지에 도움준다
 #표준화는 수치형 변수(One-Hot encoding하지 않은)에만 수행하기 때문에 
 #원-핫인코딩한 칼럼을 미리 제거한다
Pred_df_c1_norm = (Pred_df_c1 - Pred_df_c1.mean()) / Pred_df_c1.std() #표준화 적용

Pred_df_c1_res = pd.concat([Pred_df_c1_norm, ohe_time], axis=1) #표준화된 수치형 데이터와 범주형 데이터(TIME)결합.


#model1 이용
y_pred1 = model1.predict(Pred_df_c1_res) #모델이 예측 결과가 반환된다.
y_pred1 = pd.DataFrame(data = y_pred1, columns = ['FP_IDEX', 'OLD_FP_IDEX'])
 #y_pred1을 이차원 넘파이 어레이에서 데이터프레임으로 변경한다.

#Pred_df_c1_res에 값 할당
Pred_df['FP_IDEX'] = y_pred1['FP_IDEX'] #Pred_df_c1_res에 FP_IDEX 칼럼을 생성하여 y_pred1의 FP_IDEX값을 할당
Pred_df['OLD_FP_IDEX'] = y_pred1['OLD_FP_IDEX'] #Pred_df_c1_res에 OLD_FP_IDEX 칼럼을 생성하여 y_pred1의 OLD_FP_IDEX값을 할당

print(Pred_df)

In [ ]:
#전처리
Pred_df_c2 = Pred_df.copy() #전처리 전 데이터 프레임을 복제한다.

#YMD를 YEAR, MONTH, DAY로 분리하고 YEAR을 제거한다
 #YMD를 YEAR, MONTH, DAY로 분리하여 특징을 더 잘 파악할 수 있도록 한다.
Pred_df_c2['YMD'] = Pred_df_c2['YMD'].astype(str) #YMD의 타입을 문자열로 변경한다
Pred_df_c2['YEAR'] = Pred_df_c2['YMD'].str[0:4] #8자리 문자열의 0,1,2,3번째를 추출한다
Pred_df_c2['MONTH'] = Pred_df_c2['YMD'].str[4:6] #8자리 문자열의 4, 5번째를 추출한다
Pred_df_c2['DAY'] = Pred_df_c2['YMD'].str[6:8] #8자리 문자열의 6, 7번째를 추출한다
Pred_df_c2.drop('YMD', axis=1, inplace=True) #YMD는 분리하였으므로 제거한다
Pred_df_c2.drop('YEAR', axis=1, inplace=True) #년은 2020으로 일정하므로 특징이 없어서 제거한다
Pred_df_c2['MONTH'] = Pred_df_c2['MONTH'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다 
Pred_df_c2['DAY'] = Pred_df_c2['DAY'].astype(int) #딥러닝 학습에 문자열이 불가능하므로 int형으로 변경한다

#TIME이 06시 이하인 것을 제거
Pred_df_c2 = Pred_df_c2[Pred_df_c2['TIME']>6]

#TIME은 범주형 데이터이므로 One-Hot encoding을 적용한다
 #범주형 데이터가 숫자값으로 되어 학습을 오도하는 것을 방지한다
 #YMD와 TIME 모두 One-Hot encoding을 적용하면 데이터가 너무 커지기 때문에
 #둘 중 Pearson 상관계수가 큰 feature인 TIME만 One-Hot encoding을 적용한다
Pred_df_c2['TIME'] = Pred_df_c2['TIME'].astype(str) #TIME 열을 문자열 타입으로 변경한다
ohe_time = pd.get_dummies(Pred_df_c2[['TIME']]) #One-Hot encoding을 적용한다
Pred_df_c2.drop(['TIME'], axis=1, inplace=True) #표준화 적용하지 않는 칼럼(TIME) 제거

#GRID_ID One-Hot encoding 적용
Pred_df_c2['GRID_ID'] = Pred_df_c2['GRID_ID'].astype(str)
ohe_grid = pd.get_dummies(Pred_df_c2[['GRID_ID']])
Pred_df_c2.drop(['GRID_ID'], axis=1, inplace=True)

Pred_df_c2_norm = (Pred_df_c2 - Pred_df_c2.mean()) / Pred_df_c2.std() #표준화 적용

Pred_df_c2_res = pd.concat([Pred_df_c2_norm, ohe_time, ohe_grid], axis=1) #표준화된 수치형 데이터와 범주형 데이터(TIME)결합.

#model2 이용
y_pred2 = model2.predict(Pred_df_c2_res) #모델이 예측 결과가 반환된다.
y_pred2 = pd.DataFrame(data = y_pred2, columns = ['DSP_YES', 'DSP_NO'])
 #y_pred2을 이차원 넘파이 어레이에서 데이터프레임으로 변경한다.

#Pred_df_c1_res에 값 할당
Pred_df = Pred_df[Pred_df['TIME'] > 6]
Pred_df.reset_index(drop=True, inplace=True)
Pred_df['DSP_YES'] = y_pred2['DSP_YES'] #Pred_df에 DSP_YES 칼럼을 생성하여 y_pred2의 DSP_YES값을 할당
Pred_df['DSP_NO'] = y_pred2['DSP_NO'] #Pred_df에 DSP_NO 칼럼을 생성하여 y_pred2의 DSP_NO값을 할당

print(Pred_df)


In [ ]:
#EDA에서 언급한 것처럼, 1월과 0~6시는 모두 공백(0)으로 예측하고,
#나머지 격자 ID는 DSP가 있을 가장 높을 확률로 예측한다.

li = []
grid_id = [39445274, 39545254, 39545274, 39645244, 39645264]
for i in range(0, 935, 5):
  li.append(Pred_df['DSP_YES'][i])
  li.append(Pred_df['DSP_YES'][i+1])
  li.append(Pred_df['DSP_YES'][i+2])
  li.append(Pred_df['DSP_YES'][i+3])
  li.append(Pred_df['DSP_YES'][i+4])
  DSP_max = max(li)
  print(Pred_df['YMD'][i],Pred_df['TIME'][i],"의 최고 DSP_YES 값은",DSP_max,"이고 GRID_ID는",grid_id[li.index(DSP_max)],"입니다")
  li = []